In [1]:
import pandas as pd
import jpype
from konlpy.tag import Okt
import numpy as np
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from gensim.models import doc2vec
import os

c:\users\eubin\anaconda3\envs\my_env\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: This version of JPype is now deprecated. Please use this version instead: https://github.com/jpype-project/jpype
  


In [2]:
d2v_model = doc2vec.Doc2Vec.load('d2v_faqs_size30_epoch100.model')

In [3]:
faq = pd.read_csv('cleanedFaq.csv')

# 데이터프레임을 리스트로 변경
faqlist=faq['Questions'].values.tolist()

# 동의어 사전
syn_dic={'ID': ['아이디', 'id', '계정'], 
         '비밀번호': ['비밀 번호','패스워드','password','pw','비번'],
         '계좌개설': ['계좌 개설'],
         '공인인증': ['공인 인증'],
         '무상증자': ['무상 증자'],
         '유상증자': ['유상 증자'],
         '모의투자': ['모투','모의 투자'],
         '다른': ['타','XX'],
         '신주인수권부사채':['bw','신주인수권부 사채'],
         '큐트레이딩':['q-trading','qtrading','큐 트레이딩','Q-trading'],
         '주식워런트증권':['주식 워런트','주식워런트 증권','elw'],
         '오토트레이딩':['오토트레이드','auto트레이드'],
         '상하한가':['상/하한가', '상 하한가'],
         '엄브렐러펀드':['엄브렐러 펀드','엄브렐로펀드'],
         '조건형적립서비스': ['조건형','조건형 적립 서비스','조건형적립 서비스','조건형 적립서비스'],
         '연금신탁': ['연금 신탁'],
         'NH투자증권': ['엔투증','농협증권','이회사','nh투자','nh증권'],
         '로그인': ['로긴','로그 인'],
         'QVHTS': ['QV HTS','HTS','홈트레이드','홈 트레이드','홈트레이딩','홈 트레이딩', 'qv hts','hts'],
         'QVCMA': ['QV CMA', 'QV CAM', 'CMA','cma'],
         'QVMTS': ['QV MTS','MTS','나무','앱','어플','나무 어플','mts'],
         '휴대폰': ['핸드폰'],
         '분실': ['까먹','잊어버렸','잊어버림','잃어버림','잃어버렸','잊어','생각'],
         '매수': ['사고'],
         '매도': ['팔고'],
         '편': ['편리'],
         '하단': ['아래'],
         '상단': ['위'],
         '배터리': ['밧데리','빠때리','빠떼리'],
         '브라우저': ['browser','크롬','익스플로러','chrome','explorer'],
         '이용': ['사용'],
         '외국어': ['영어','중국어','영문']
        }

# 동의어 사전으로 바꿔주는 함수
def replace_syn(clean_sentence): 
    normalised_sentence = [] 
    # 소문자 바꾸기 
    for word in clean_sentence: 
        normalised = [list(syn_dic.keys())[i] for i in range(len(list(syn_dic.values()))) if word in list(syn_dic.values())[i]] 

        if len(normalised) > 0: 
            normalised_sentence.append(normalised[0]) 
        else: 
            normalised_sentence.append(word) 
             
    return normalised_sentence 

okt=Okt()

# 특정 품사 제외한 후 동의어 대체하는 함수
def tokenize_mc(doc):
    #jpype.attachThreadToJVM()
    mailist=okt.pos(doc)
    r=[]
    for word, pumsa in mailist:
        if not pumsa in ['Josa','Punctuation','Verb','Adjective']:
            r.append(word)
        token_doc=replace_syn(r)
    return token_doc
# SF: 마침표,물음표,느낌표
# SSO, SSC: 여는 괄호, 닫는 괄호
# SC: 구분자
# JX: 보조사

token_faqs=[]
for sentence in faqlist:
    temp_X=tokenize_mc(sentence)
    token_faqs.append(temp_X)

tagged_faqs = [TaggedDocument(doc, [i]) for i, doc in enumerate(token_faqs)]

In [4]:
words_dic={'비밀번호': [556, 129, 355], #0
          '공인인증': [16, 13, 10], #1
          '계좌개설': [77, 638, 41], #2
          '펀드': [273, 494, 596], #3
          '모의투자': [95, 97, 98], #4
          '주식': [322, 535, 341], #5
          'HTS': [445, 363, 357], #6
          'CMA': [194, 123, 75], #7
          'MTS': [629, 618, 620]} #8

In [5]:
def chatbot():
    while True:
        print('안녕하세요, 언제나 당신 옆에 있는 저는 엔투봇입니다.\n')
        input_text=input('질문 혹은 단어를 입력해주세요: ')
        #print('단어 예시) 비밀번호 / 공인인증 / 계좌개설\n'+
                        #'질문 예시) 신청해두면 편리한 서비스를 알려주세요')
        #input_text=tokenize_mc(input_text)
        dummy=input_text
        if dummy in list(words_dic.keys()):
            if dummy==list(words_dic.keys())[0]:
                print('다음 제시된 목록 중 궁금하신 내용이 있으신가요?\n'+
                '1. 비밀번호 변경 2. 비밀번호 종류 3. 거래비밀번호 저장 기능 4.원하는 질문이 없어요')
            elif dummy==list(words_dic.keys())[1]:
                print('다음 제시된 목록 중 궁금하신 내용이 있으신가요?\n'+
                '1. 공인인증서 재발급 2. 공인인증서 유효기간 3. 방화벽 환경 4.원하는 질문이 없어요')
            elif dummy==list(words_dic.keys())[2]:
                print('다음 제시된 목록 중 궁금하신 내용이 있으신가요?\n'+
                '1. 필요한 서류 2. 계좌개설 방법 3. 대리인 개설 4.원하는 질문이 없어요')
            elif dummy==list(words_dic.keys())[3]:
                print('다음 제시된 목록 중 궁금하신 내용이 있으신가요?\n'+
                '1. 펀드의 종류 2. 펀드 환매 3. 연금저축펀드 해지 4.원하는 질문이 없어요')
            elif dummy==list(words_dic.keys())[4]:
                print('다음 제시된 목록 중 궁금하신 내용이 있으신가요?\n'+
                '1. 모의투자 신청 방법 2. 기간 연장 3. 투자금액 변경 4.원하는 질문이 없어요')
            elif dummy==list(words_dic.keys())[5]:
                print('다음 제시된 목록 중 궁금하신 내용이 있으신가요?\n'+
                '1. 주식 대체 서비스 2. 해외주식 매매방법 3. 타사 대체 입고 4.원하는 질문이 없어요')
            elif dummy==list(words_dic.keys())[6]:
                print('다음 제시된 목록 중 궁금하신 내용이 있으신가요?\n'+
                '1. 단축키 등록 방법 2. 초기 화면 지정 방법 3. 관심종목 조건 설정 4.원하는 질문이 없어요')
            elif dummy==list(words_dic.keys())[7]:
                print('다음 제시된 목록 중 궁금하신 내용이 있으신가요?\n'+
                '1. CMA서비스란 2. 거래 가능한 상품 3. 자동투자 상품 4.원하는 질문이 없어요')
            else:
                print('다음 제시된 목록 중 궁금하신 내용이 있으신가요?\n'+
                '1. 주식 선물 매매 2. 관심종목 안보임 3. 알림서비스 4.원하는 질문이 없어요')
            
            input2=input('해당되는 번호를 골라주세요: ')
            value_idx=int(input2)
                #input2=str(input2)
            if value_idx<4:
                print(faq['Answers'][words_dic[input_text][value_idx-1]-1])
                break
            else:
                print('1544-0000으로 전화주시면 친절히 안내해드리겠습니다 ^^')
                break


        else:
            input_text2=tokenize_mc(input_text)
            test_vector = d2v_model.infer_vector(input_text2, epochs=100)
            result = d2v_model.docvecs.most_similar([test_vector], topn=1)
            print(result)
            if result[0][1]>0.67:
                print('예상한 질문: ' + faq['Questions'][result[0][0]])
                print('-'*100)
                print(faq['Answers'][result[0][0]])
                break
            else:
                print('아직 제가 알지 못하는 내용이에요. 1544-0000으로 전화주시면 친절히 안내해드리겠습니다 ^^')
                break



In [7]:
chatbot()

안녕하세요, 언제나 당신 옆에 있는 저는 엔투봇입니다.

질문 혹은 단어를 입력해주세요: 아이디 변경
[(0, 0.8446037173271179)]
예상한 질문:ID를 변경하고 싶습니다.
----------------------------------------------------------------------------------------------------
A. 
사용중이신 ID를 변경하려면, 기존에 사용중인 ID를 해지하신 후 원하는 ID로 재등록 하시기 바랍니다. 

[ID해지 방법] 
방법1. 홈페이지,MTS에서 본인 확인절차를 거친 후 ID 해지 
① 홈페이지 : 뱅킹계좌정보 > 계좌개인정보관리 > ID개인정보 > ID해지 
② MTS : 안내문의등록설정 > 개인정보서비스 > ID비밀번호 관리 > ID해지 

방법2. 가까운 영업점을 방문하여 ID해지 신청
※ ID해지시 구비서류 
* 본인 : 실명확인증표, 거래인감(자필서명 가능) 
* 대리인 : 본인대리인 실명확인증표, 거래인감, 위임장 

방법3. 고객지원센터(☎15440000)로 전화하여 ID해지 신청  단, 개인고객만 유선(전화)로 ID해지 신청 가능합니다. (법인은 영업점 내방하여 ID해지하셔야 합니다)



